In [1]:
!pip install transformers
!pip install sumy
!pip install spacy
!python -m spacy download en_core_web_sm



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python3 -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 54.9 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:

import pandas as pd
import json
import spacy
from collections import defaultdict
from transformers import pipeline
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
# Ensure the language model is downloaded
!python -m spacy download en_core_web_sm
from tqdm import tqdm
from typing import List, Dict
from sumy.nlp.tokenizers import Tokenizer
# Load your dataset
df = pd.read_csv("sample_10.csv")

# Load models
nlp = spacy.load("en_core_web_sm")
summarizer = pipeline("summarization", model="google/pegasus-xsum", tokenizer="google/pegasus-xsum")
ner_model = pipeline("ner", model="d4data/biomedical-ner-all", tokenizer="d4data/biomedical-ner-all", aggregation_strategy="simple")

# Utilities
def preprocess_input(text):
    return text.replace('\n', ' ')[:1024]

def abstractive_summary(text):
    try:
        result = summarizer(preprocess_input(text), max_length=150, min_length=40, do_sample=False)
        return result[0]['summary_text']
    except:
        return ""

def extractive_summary(text, n=3):
    try:
        parser = PlaintextParser.from_string(text, Tokenizer("english"))
        summarizer_obj = TextRankSummarizer()
        summary = summarizer_obj(parser.document, n)
        return " ".join(str(sentence) for sentence in summary)
    except:
        return ""

def extract_entities_structured(text):
    try:
        entities = ner_model(text[:512])
        entity_dict = defaultdict(list)
        for ent in entities:
            entity_dict[ent['entity_group']].append(ent['word'])
        return dict(entity_dict)
    except:
        return {}

def extract_age(text):
    for ent in nlp(text).ents:
        if ent.label_ == "DATE" and "year" in ent.text:
            return ent.text
    return None

# Safely parse the JSON summary
def smart_parse_summary(summary_str):
    try:
        # Try direct parsing
        parsed = json.loads(summary_str)

        # Now fix: if parsed is a dictionary, and "visit motivation" is another dictionary
        vm = parsed.get("visit motivation")
        if isinstance(vm, dict):
            parsed["visit motivation"] = vm.get("visit motivation", "- N/A")

        return parsed

    except Exception as e:
        print(f"Parsing Error: {e}")
        return {
            "visit motivation": "- N/A",
            "patient information": {"age": "N/A", "sex": "N/A"},
            "diagnosis tests": [],
            "treatments": []
        }


# ✨ NEW: Format into structured clinical text
def clean_formatted_summary(parsed_summary):
    pi = parsed_summary.get("patient information", {})
    vm = parsed_summary.get("visit motivation", "- N/A")
    diagnosis_tests = parsed_summary.get("diagnosis tests", [])
    treatments = parsed_summary.get("treatments", [])

    diagnoses = []
    meds = []
    txs = []

    if isinstance(diagnosis_tests, list):
        for d in diagnosis_tests:
            if isinstance(d, dict) and d.get("condition") and d.get("condition") != "None":
                diagnoses.append(f"- {d['condition']}")

    if isinstance(treatments, list):
        for t in treatments:
            if isinstance(t, dict):
                if t.get("name") and t.get("name") != "None" and t.get("dosage") and t.get("frequency"):
                    meds.append(f"- {t['name']} {t['dosage']} {t['frequency']}")
                if t.get("details") and t.get("details") != "None":
                    txs.append(f"- {t['details']}")

    age = pi.get('age', "N/A")
    sex = pi.get('sex', "N/A")

    # Fix if nested visit motivation
    if isinstance(vm, dict):
        vm = vm.get("visit motivation", "- N/A")

    summary_lines = [
        "CLINICAL SUMMARY:",
        f"Patient: {age} {sex.lower() if sex else ''}",
        ""
    ]

    if vm and vm != "- N/A":
        summary_lines.extend(["VISIT MOTIVATION:", vm, ""])

    if diagnoses:
        summary_lines.extend(["KEY DIAGNOSES:", *diagnoses, ""])

    if meds:
        summary_lines.extend(["MEDICATIONS:", *meds, ""])

    if txs:
        summary_lines.extend(["TREATMENTS:", *txs, ""])

    return "\n".join(summary_lines).strip()



# 🔥 Now run this loop for ALL 10 rows:


/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


FileNotFoundError: [Errno 2] No such file or directory: 'sample_10.csv'

In [ ]:
# ✅ Install tqdm if needed
# !pip install tqdm

import pandas as pd
import json
import spacy
from collections import defaultdict
from transformers import pipeline
from tqdm import tqdm

# Load data
df_aug = pd.read_csv("augmented_notes_30K.csv")

# Load models
nlp_aug = spacy.load("en_core_web_sm")
summarizer_aug = pipeline("summarization", model="google/pegasus-xsum", tokenizer="google/pegasus-xsum")
ner_model_aug = pipeline("ner", model="d4data/biomedical-ner-all", aggregation_strategy="simple")

# Parse summaries
def smart_parse_summary_aug(summary_str):
    try:
        summary_str = summary_str.replace('\x00', '').replace('\x1f', '').replace('\x1e', '')
        parsed = json.loads(summary_str)

        vm = parsed.get("visit motivation")
        if isinstance(vm, dict):
            parsed["visit motivation"] = vm.get("visit motivation", "- N/A")

        return parsed

    except Exception as e:
        print(f"Parsing Error: {e}")
        return {
            "visit motivation": "- N/A",
            "patient information": {"age": "N/A", "sex": "N/A"},
            "diagnosis tests": [],
            "treatments": []
        }

# Format structured output
def clean_formatted_summary_aug(parsed_summary):
    pi = parsed_summary.get("patient information", {})

    if isinstance(pi, list):
        if len(pi) > 0 and isinstance(pi[0], dict):
            pi = pi[0]
        else:
            pi = {}

    vm = parsed_summary.get("visit motivation", "- N/A")
    diagnosis_tests = parsed_summary.get("diagnosis tests", [])
    treatments = parsed_summary.get("treatments", [])

    diagnoses = []
    meds = []
    txs = []

    if isinstance(diagnosis_tests, list):
        for d in diagnosis_tests:
            if isinstance(d, dict) and d.get("condition") and d.get("condition") != "None":
                diagnoses.append(f"- {d['condition']}")

    if isinstance(treatments, list):
        for t in treatments:
            if isinstance(t, dict):
                if t.get("name") and t.get("name") != "None" and t.get("dosage") and t.get("frequency"):
                    meds.append(f"- {t['name']} {t['dosage']} {t['frequency']}")
                if t.get("details") and t.get("details") != "None":
                    txs.append(f"- {t['details']}")

    age = pi.get('age', "N/A")
    sex = pi.get('sex', "N/A")

    if isinstance(vm, dict):
        vm = vm.get("visit motivation", "- N/A")
    elif isinstance(vm, list):
        vm = " ".join(vm) if vm else "- N/A"

    summary_lines = [
        "CLINICAL SUMMARY:",
        f"Patient: {age} {sex.lower() if sex else ''}",
        ""
    ]

    if vm and vm != "- N/A":
        summary_lines.extend(["VISIT MOTIVATION:", vm, ""])

    if diagnoses:
        summary_lines.extend(["KEY DIAGNOSES:", *diagnoses, ""])

    if meds:
        summary_lines.extend(["MEDICATIONS:", *meds, ""])

    if txs:
        summary_lines.extend(["TREATMENTS:", *txs, ""])

    return "\n".join(summary_lines).strip()

# Parse safely first
df_aug['parsed_summary_aug'] = df_aug['summary'].apply(smart_parse_summary_aug)

# Prepare final outputs
final_outputs_aug = []

# Use tqdm to track progress
for idx, row in tqdm(df_aug.iterrows(), total=len(df_aug), desc="Processing rows"):
    parsed_summary = row['parsed_summary_aug']
    clean_output = clean_formatted_summary_aug(parsed_summary)

    final_outputs_aug.append({
        "index": idx,
        "final_summary": clean_output
    })

    # Batch save every 5000 rows
    if (idx + 1) % 5000 == 0:
        batch_df = pd.DataFrame(final_outputs_aug)
        batch_df.to_csv(f"batch_cleaned_summaries_up_to_row_{idx+1}.csv", index=False)
        print(f"✅ Saved intermediate file at row {idx+1}")

# Final full save
final_df_aug = pd.DataFrame(final_outputs_aug)
final_df_aug.to_csv("final_cleaned_summaries_augmented30k.csv", index=False)

print("✅ All clean summaries saved to 'final_cleaned_summaries_augmented30k.csv'!")

# Print first 20 only for checking
for out in final_outputs_aug[:20]:
    print(f"===== Summary for Row {out['index']} =====")
    print(out['final_summary'])
    print("\n\n")


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
Device set to use cuda:0


Streaming output truncated to the last 5000 lines.
Parsing Error: Invalid control character at: line 54 column 85 (char 1726)
Parsing Error: Invalid control character at: line 72 column 56 (char 1640)
Parsing Error: Invalid control character at: line 40 column 137 (char 1194)
Parsing Error: Invalid control character at: line 112 column 168 (char 2935)
Parsing Error: Invalid control character at: line 39 column 54 (char 1128)
Parsing Error: Invalid control character at: line 68 column 88 (char 2132)
Parsing Error: Expecting ',' delimiter: line 20 column 17 (char 472)
Parsing Error: Invalid control character at: line 40 column 163 (char 1188)
Parsing Error: Invalid control character at: line 83 column 37 (char 2111)
Parsing Error: Invalid control character at: line 87 column 73 (char 2499)
Parsing Error: Invalid control character at: line 78 column 154 (char 2559)
Parsing Error: Invalid control character at: line 74 column 154 (char 2186)
Parsing Error: Invalid control character at: line

Processing rows:  29%|██▉       | 8797/30000 [00:00<00:01, 15162.97it/s]

✅ Saved intermediate file at row 5000


Processing rows:  41%|████      | 12313/30000 [00:00<00:01, 14564.74it/s]

...

Processing rows:  92%|█████████▏| 27697/30000 [00:02<00:00, 7141.25it/s]

✅ Saved intermediate file at row 25000


Processing rows: 100%|██████████| 30000/30000 [00:03<00:00, 8777.28it/s]

✅ Saved intermediate file at row 30000


✅ All clean summaries saved to 'final_cleaned_summaries_augmented30k.csv'!
===== Summary for Row 0 =====
CLINICAL SUMMARY:
Patient: Sixteen years old female

VISIT MOTIVATION:
Discomfort in the neck and lower back, restriction of body movements, inability to maintain an erect posture, and requiring assistance in standing and walking.

MEDICATIONS:
- Olanzapine tablets 5 mg per day Daily
- Trihexyphenidyl 4 mg per day Daily

TREATMENTS:
- Previously managed with olanzapine tablets in 2.5–10 mg doses per day at different times over the past seven years.



===== Summary for Row 1 =====
CLINICAL SUMMARY:
Patient: N/A n/a



===== Summary for Row 2 =====
CLINICAL SUMMARY:
Patient: 36 years old female

VISIT MOTIVATION:
Pain and restricted range of motion in the left hip joint

KEY DIAGNOSES:
- Idiopathic osteonecrosis of the femoral head



===== Summary for Row 3 =====
CLINICAL SUMMARY:
Patient: 49 male

VISIT MOTIVATION:
Pain in the left proximal forearm after a fall

KEY DIAGNOSES:
- Pr

In [ ]:
df1 = pd.read_json("hf://datasets/AGBonnet/augmented-clinical-notes/augmented_notes_30K.jsonl", lines=True)

In [ ]:
# prompt: convert this df1 to csv file

import pandas as pd

df1 = pd.read_json("hf://datasets/AGBonnet/augmented-clinical-notes/augmented_notes_30K.jsonl", lines=True)
df1.to_csv('augmented_notes_30K.csv', index=False)


In [ ]:
# ✅ Install first if needed
# !pip install transformers sumy spacy
# !python -m spacy download en_core_web_sm

# Imports
import pandas as pd
import json
import spacy
from collections import defaultdict
from transformers import pipeline
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

# Load the 30K augmented clinical notes dataset
df_aug = pd.read_csv("augmented_notes_30K.csv")

# Load models
nlp_aug = spacy.load("en_core_web_sm")
summarizer_aug = pipeline("summarization", model="google/pegasus-xsum", tokenizer="google/pegasus-xsum")
ner_model_aug = pipeline("ner", model="d4data/biomedical-ner-all", tokenizer="d4data/biomedical-ner-all", aggregation_strategy="simple")

# Utilities
def preprocess_input_aug(text):
    return text.replace('\n', ' ')[:1024]

def abstractive_summary_aug(text):
    try:
        result = summarizer_aug(preprocess_input_aug(text), max_length=150, min_length=40, do_sample=False)
        return result[0]['summary_text']
    except:
        return ""

def extractive_summary_aug(text, n=3):
    try:
        parser = PlaintextParser.from_string(text, Tokenizer("english"))
        summarizer_obj = TextRankSummarizer()
        summary = summarizer_obj(parser.document, n)
        return " ".join(str(sentence) for sentence in summary)
    except:
        return ""

def extract_entities_structured_aug(text):
    try:
        entities = ner_model_aug(text[:512])
        entity_dict = defaultdict(list)
        for ent in entities:
            entity_dict[ent['entity_group']].append(ent['word'])
        return dict(entity_dict)
    except:
        return {}

def extract_age_aug(text):
    for ent in nlp_aug(text).ents:
        if ent.label_ == "DATE" and "year" in ent.text:
            return ent.text
    return None

# Parse the JSON summary smartly
def smart_parse_summary_aug(summary_str):
    try:
        parsed = json.loads(summary_str)

        # Fix if "visit motivation" itself is a dictionary
        vm = parsed.get("visit motivation")
        if isinstance(vm, dict):
            parsed["visit motivation"] = vm.get("visit motivation", "- N/A")

        return parsed

    except Exception as e:
        print(f"Parsing Error: {e}")
        return {
            "visit motivation": "- N/A",
            "patient information": {"age": "N/A", "sex": "N/A"},
            "diagnosis tests": [],
            "treatments": []
        }

# Format into structured clinical text (only real sections)
def clean_formatted_summary_aug(parsed_summary):
    pi = parsed_summary.get("patient information", {})
    vm = parsed_summary.get("visit motivation", "- N/A")
    diagnosis_tests = parsed_summary.get("diagnosis tests", [])
    treatments = parsed_summary.get("treatments", [])

    diagnoses = []
    meds = []
    txs = []

    if isinstance(diagnosis_tests, list):
        for d in diagnosis_tests:
            if isinstance(d, dict) and d.get("condition") and d.get("condition") != "None":
                diagnoses.append(f"- {d['condition']}")

    if isinstance(treatments, list):
        for t in treatments:
            if isinstance(t, dict):
                if t.get("name") and t.get("name") != "None" and t.get("dosage") and t.get("frequency"):
                    meds.append(f"- {t['name']} {t['dosage']} {t['frequency']}")
                if t.get("details") and t.get("details") != "None":
                    txs.append(f"- {t['details']}")

    age = pi.get('age', "N/A")
    sex = pi.get('sex', "N/A")

    if isinstance(vm, dict):
        vm = vm.get("visit motivation", "- N/A")

    summary_lines = [
        "CLINICAL SUMMARY:",
        f"Patient: {age} {sex.lower() if sex else ''}",
        ""
    ]

    if vm and vm != "- N/A":
        summary_lines.extend(["VISIT MOTIVATION:", vm, ""])

    if diagnoses:
        summary_lines.extend(["KEY DIAGNOSES:", *diagnoses, ""])

    if meds:
        summary_lines.extend(["MEDICATIONS:", *meds, ""])

    if txs:
        summary_lines.extend(["TREATMENTS:", *txs, ""])

    summary_lines.extend(["FOLLOW-UP:", "4 weeks for reassessment"])

    return "\n".join(summary_lines).strip()

# Step 1: Parse the 'summary' column properly for all rows
df_aug['parsed_summary_aug'] = df_aug['summary'].apply(smart_parse_summary_aug)

# Step 2: Generate cleaned final summaries
final_outputs_aug = []

for idx, row in df_aug.iterrows():
    parsed_summary = row['parsed_summary_aug']
    clean_output = clean_formatted_summary_aug(parsed_summary)

    final_outputs_aug.append({
        "index": idx,
        "final_summary": clean_output
    })

# Step 3: Save all summaries into a new CSV file
final_df_aug = pd.DataFrame(final_outputs_aug)
final_df_aug.to_csv("final_cleaned_summaries_augmented30k.csv", index=False)

print("✅ Final clean summaries saved to 'final_cleaned_summaries_augmented30k.csv'!")

# Step 4: Also print a few samples nicely
for out in final_outputs_aug[:5]:  # printing only first 5 to avoid huge output
    print(f"===== Summary for Row {out['index']} =====")
    print(out['final_summary'])
    print("\n\n")


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
Device set to use cuda:0


Streaming output truncated to the last 5000 lines.
Parsing Error: Invalid control character at: line 54 column 85 (char 1726)
Parsing Error: Invalid control character at: line 72 column 56 (char 1640)
Parsing Error: Invalid control character at: line 40 column 137 (char 1194)
Parsing Error: Invalid control character at: line 112 column 168 (char 2935)
Parsing Error: Invalid control character at: line 39 column 54 (char 1128)
Parsing Error: Invalid control character at: line 68 column 88 (char 2132)
Parsing Error: Expecting ',' delimiter: line 20 column 17 (char 472)
Parsing Error: Invalid control character at: line 40 column 163 (char 1188)
Parsing Error: Invalid control character at: line 83 column 37 (char 2111)
Parsing Error: Invalid control character at: line 87 column 73 (char 2499)
Parsing Error: Invalid control character at: line 78 column 154 (char 2559)
Parsing Error: Invalid control character at: line 74 column 154 (char 2186)
Parsing Error: Invalid control character at: line

AttributeError: 'list' object has no attribute 'get'